In [ ]:
import numpy as np
import pandas as pd
import sklearn
import os

In [ ]:
source_file = "./data/combined_data.csv"
destination_file = "./data"



## k-mer Nucleotide Frequency

## k-Spaced Nucleotide Pair Frequency

## 